The SegNet VGG 16 model notebook
cited from:

@article{badrinarayanan2017segnet,
  title={Segnet: A deep convolutional encoder-decoder architecture for image segmentation},
  author={Badrinarayanan, Vijay and Kendall, Alex and Cipolla, Roberto},
  journal={IEEE transactions on pattern analysis and machine intelligence},
  volume={39},
  number={12},
  pages={2481--2495},
  year={2017},
  publisher={IEEE}
}

In [1]:

%matplotlib inline
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pandas as pd
from glob import glob
import pickle
import tensorflow as tf

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="6DUZXUzEElvb164kQwwY")
project = rf.workspace("pingworkspace").project("txlpbc-v2-vnadh")
version = project.version(1)
dataset = version.download("voc")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 102.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 124.0 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


In [ ]:
data_path = 'txlpbc-v2-1/'

train_path = "txlpbc-v2-1/train/"

valid_path = "txlpbc-v2-1/valid/"

test_path = "txlpbc-v2-1/test/"

# train_file = data_path + "train.p"
# valid_file = data_path + "val.p"
# test_file = data_path + "test.p"

In [ ]:
import os
import cv2
import xml.etree.ElementTree as ET
import numpy as np

def class_name_to_id(class_name):
    class_map = {
        'PC': 0,  # Replace with actual class names
        'RBC': 1,
        'WBC': 2,
    }
    return class_map.get(class_name, -1)  # Default to -1 for unknown classes

def parse_xml(xml_file, image_shape, num_classes=3):
    # Initialize mask (2D) where 0 represents background
    mask = np.zeros(image_shape[:2], dtype=np.uint8)
    tree = ET.parse(xml_file)
    root = tree.getroot()

    for obj in root.findall('object'):
        class_name = obj.find('name').text
        class_id = class_name_to_id(class_name)  # Assuming function to map class names to ids

        polygon = obj.find('polygon')
        if polygon is not None:
            # Extract polygon points
            points = []
            i = 1
            while True:
                x = polygon.find(f'x{i}')
                y = polygon.find(f'y{i}')
                if x is None or y is None:
                    break  # Exit if no more points
                points.append([int(float(x.text)), int(float(y.text))])
                i += 1

            if points:
                # Convert points list to a proper format for OpenCV (Nx2 array)
                points = np.array(points, dtype=np.int32)
                points = points.reshape((-1, 1, 2))  # Reshape to required format for fillPoly

                # Fill the polygon on the mask with the class ID
                cv2.fillPoly(mask, [points], class_id)

        else:
            # Fallback to bounding box if polygon is not available
            bbox = obj.find('bndbox')
            if bbox is not None:
                xmin = int(bbox.find('xmin').text)
                ymin = int(bbox.find('ymin').text)
                xmax = int(bbox.find('xmax').text)
                ymax = int(bbox.find('ymax').text)

                # Draw class_id into the bounding box area of the mask
                mask[ymin:ymax, xmin:xmax] = class_id

    return mask

def make_dataset(image_dir, num_classes=3):
    images = []
    masks = []

    for filename in os.listdir(image_dir):
        if filename.endswith('.jpg'):
            image_path = os.path.join(image_dir, filename)
            xml_path = os.path.join(image_dir, filename[:-4] + '.xml')  # Use the same filename for the XML

            # Read and resize the image to 224x224
            image = cv2.resize(cv2.imread(image_path), (224, 224))  # Assuming input size is 224x224
            images.append(image)

            # Parse the corresponding XML file to create the mask
            mask = parse_xml(xml_path, image.shape, num_classes)
            mask = cv2.resize(mask, (224, 224), interpolation=cv2.INTER_NEAREST)
            masks.append(mask)

    images = np.array(images)
    masks = np.array(masks)
    return images, masks


# Directory containing your dataset
# image_dir = 'txlpbc-v2-1/train'
# X_train, y_train = make_dataset(image_dir, num_classes=3)


In [ ]:
X_train, y_train = make_dataset(train_path)
X_val, y_val = make_dataset(valid_path)
X_test, y_test = make_dataset(test_path)

In [ ]:
X_train = np.concatenate([X_train, X_val])
y_train = np.concatenate([y_train, y_val])

In [ ]:
print(X_train.shape)
print(y_train.shape)

In [ ]:
from matplotlib.gridspec import GridSpec
from random import randint, sample
ranidx = randint(0, len(y_train))
gs = GridSpec(4,4)
plt.figure(dpi=100)
for i in range(3):
    plt.subplot(gs[i]), plt.imshow(y_train[ranidx][:,:]), plt.axis('off')

plt.subplot(gs[3]), plt.imshow(X_train[ranidx]), plt.axis('off')

In [ ]:
import xml.etree.ElementTree as ET
#RBC = [128,128,128]
RBC = [128,0,0]
# Building = [128,0,0]
# Pole = [192,192,128]
# Road_marking = [255,69,0]
# Road = [128,64,128]
# Pavement = [60,40,222]
# WBC = [128,128,0]
WBC = [60,40,222]
# SignSymbol = [192,128,128]
# Fence = [64,64,128]
PC = [64,0,128]
# Pedestrian = [64,64,0]
# Bicyclist = [0,128,192]
Unlabelled = [0,0,0]

#label_colours = np.array([RBC, WBC, PC, Unlabelled])
label_colours = np.array([Unlabelled,RBC, WBC, PC])
def visualize(temp, plot=True):
    r = temp.copy()
    g = temp.copy()
    b = temp.copy()

    for l in [0, 1, 2]:
        r[temp==l]=label_colours[l,0]
        g[temp==l]=label_colours[l,1]
        b[temp==l]=label_colours[l,2]

    rgb = np.zeros((temp.shape[0], temp.shape[1], 3))
    rgb[:,:,0] = (r/255.0)#[:,:,0]
    rgb[:,:,1] = (g/255.0)#[:,:,1]
    rgb[:,:,2] = (b/255.0)#[:,:,2]
    if plot:
        plt.imshow(rgb)
    else:
        return rgb

# randsample = sample(range(0,len(pred)), 4)
randsample = [0, 1, 2, 3]
gs = GridSpec(3,5)
plt.figure(dpi=200)

for i in range(4):
    ground_truth_path = y_test[randsample[i]]  # Path to XML ground truth file
    ground_truth_labels = parse_xml_label(ground_truth_path)
    ground_truth_visual = visualize(ground_truth_labels, plot=False)
    plt.subplot(gs[10 + i]), plt.imshow(ground_truth_visual), plt.axis('off')

    plt.subplot(gs[i]), plt.imshow(X_test[randsample[i]]), plt.axis('off')
    plt.subplot(gs[5+i]), plt.imshow(output), plt.axis('off')
    #plt.subplot(gs[10+i]), plt.imshow(out_test), plt.axis('off')

In [ ]:
print(X_train.shape[1:] )

In [ ]:
num_classes = 3
input_shape = X_train.shape[1:] #(480,480,3)
smooth = 1.
nproc = 8

In [ ]:
pip install --user keras

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Activation, BatchNormalization
from keras.optimizers import Adam

# Create SegNet model for 3 classes
def segnet(input_shape=(224, 224, 3), num_classes=3):
    inputs = Input(shape=input_shape)

    # Encoder (same as VGG16)
    x = Conv2D(64, (3, 3), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(128, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(128, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(256, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(256, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(256, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(512, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(512, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(512, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(512, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(512, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(512, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # Decoder
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(512, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(512, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(512, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(512, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(512, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(256, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(256, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(256, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(128, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(128, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Final layer
    outputs = Conv2D(num_classes, (1, 1), padding='same')(x)
    outputs = Activation('softmax')(outputs)  # Use softmax for multi-class classification

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Create the model
model = segnet(input_shape=(224, 224, 3), num_classes=3)
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print(model.summary())


In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=8,
                    epochs=200,
                    shuffle=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
history.history.keys()

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
pred = model.predict(X_test, verbose=1)

ranidx = randint(0, len(y_test))
gs = GridSpec(1,12)
plt.figure(dpi=200)
for i in range(12):
    plt.subplot(gs[i]), plt.imshow(pred[ranidx][:,:], cmap='jet'), plt.axis('off')

plt.figure(dpi=200)
for i in range(12):
    plt.subplot(gs[i]), plt.imshow(y_test[ranidx][:,:],cmap='jet'), plt.axis('off')

plt.figure(),plt.imshow(X_test[ranidx]), plt.axis('off')

In [ ]:
# refer from : https://github.com/preddy5/segnet/blob/master/Segnet.ipynb
#RBC = [128,128,128]
RBC = [128,0,0]
# Building = [128,0,0]
# Pole = [192,192,128]
# Road_marking = [255,69,0]
# Road = [128,64,128]
# Pavement = [60,40,222]
# WBC = [128,128,0]
WBC = [60,40,222]
# SignSymbol = [192,128,128]
# Fence = [64,64,128]
PC = [64,0,128]
# Pedestrian = [64,64,0]
# Bicyclist = [0,128,192]
Unlabelled = [0,0,0]

#label_colours = np.array([RBC, WBC, PC, Unlabelled])
label_colours = np.array([Unlabelled,RBC, WBC, PC])
def visualize(temp, plot=True):
    r = temp.copy()
    g = temp.copy()
    b = temp.copy()

    for l in [0, 1, 2]:
        r[temp==l]=label_colours[l,0]
        g[temp==l]=label_colours[l,1]
        b[temp==l]=label_colours[l,2]

    rgb = np.zeros((temp.shape[0], temp.shape[1], 3))
    rgb[:,:,0] = (r/255.0)#[:,:,0]
    rgb[:,:,1] = (g/255.0)#[:,:,1]
    rgb[:,:,2] = (b/255.0)#[:,:,2]
    if plot:
        plt.imshow(rgb)
    else:
        return rgb

# randsample = sample(range(0,len(pred)), 4)
randsample = [0, 1, 2, 3]
gs = GridSpec(3,5)
plt.figure(dpi=200)

for i in range(4):
    output = visualize(np.argmax(pred[randsample[i]],axis=2).reshape((224,224)), plot=False)
    #out_test = visualize(np.argmax(y_test[randsample[i]],axis=2).reshape((224,224)), plot=False)

    plt.subplot(gs[i]), plt.imshow(X_test[randsample[i]]), plt.axis('off')
    plt.subplot(gs[5+i]), plt.imshow(output), plt.axis('off')
    #plt.subplot(gs[10+i]), plt.imshow(out_test), plt.axis('off')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

# Color mappings
RBC = [128, 0, 0]
WBC = [60, 40, 222]
PC = [64, 0, 128]
Unlabelled = [0, 0, 0]
label_colours = np.array([Unlabelled, RBC, WBC, PC])

# Visualization function
def visualize(temp, plot=True):
    r = temp.copy()
    g = temp.copy()
    b = temp.copy()

    for l in [0, 1, 2, 3]:  # Updated to include all labels in label_colours
        r[temp == l] = label_colours[l, 0]
        g[temp == l] = label_colours[l, 1]
        b[temp == l] = label_colours[l, 2]

    rgb = np.zeros((temp.shape[0], temp.shape[1], 3))
    rgb[:, :, 0] = r / 255.0
    rgb[:, :, 1] = g / 255.0
    rgb[:, :, 2] = b / 255.0
    if plot:
        plt.imshow(rgb)
    else:
        return rgb

# Sample data
randsample = [0, 1, 2, 3]
gs = GridSpec(3, 5)
plt.figure(dpi=200)

for i in range(4):
    # Display original image
    plt.subplot(gs[i]), plt.imshow(X_test[randsample[i]]), plt.axis('off')

    # Display model prediction
    output = visualize(np.argmax(pred[randsample[i]], axis=2).reshape((224, 224)), plot=False)
    plt.subplot(gs[5 + i]), plt.imshow(output), plt.axis('off')

    # Display ground truth from y_test
    ground_truth_visual = visualize(y_test[randsample[i]], plot=False)
    plt.subplot(gs[10 + i]), plt.imshow(ground_truth_visual), plt.axis('off')


In [ ]:
model.save('txlpbc-v2-1/blood_cell_segnet_model.h5')